In [2]:
from dao.mongodbdao import MongoDBDao
from tfidf.TF_IDF import TF_IDF
import numpy as np 

dao = MongoDBDao({
    "host": "127.0.0.1",
    "db": "default",
    "wordindex_collection": "wordindex",
    "pagedetails_collection": "pagedetails",
    "username": "root",
    "password": "spezanw"
})

In [2]:
tfidf = TF_IDF(dao)

TF-IDF module: Initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.276386.php'} contains no words
TF-IDF module: Initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.461948.php'} contains no words
TF-IDF module: Initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.587702.php'} contains no words
TF-IDF module: Initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.85381.php'} contains no words
TF-IDF module: Inital TF-IDF calculation: no idf for {161139} terms


# diverse Tests

In [3]:
a = [[1, 2],[3, 4]]
for j in a:
    print(j)

[1, 2]
[3, 4]


In [4]:
term_count = dao.getAllWordsWithCounts()
print("droge" in term_count)

True


In [5]:
#tfidf.getResultsForInput(user_input=["fußgängerin", "gestern", "nachmittag", "polizei", "waffe", "drogen", "fahrrad"], n_results=100, return_sims=True)
#tfidf.tf_idfs["https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.247355.php"]

tfidf.getResultsForInput(user_input=["drogen"], n_results=100, return_sims=True)



TF-IDF module: Input TF-IDF calculation: word {'drogen'} in query is not in db


[]

TF: 
- für jedes Dokument alle Wörter

IDF:
- Anzahl der Dokumente
- alle Wörter
- in wie vielen Dokumenten kommt jedes Wort vor?

1. Index: Anzahl der Dokumente
2. Inverser Index: Für term t get t.length

In [6]:
test_url = "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.247355.php"
test_url2 = "https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.408478.php"

dao.getPageDetails(test_url2)

{'_id': 'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.408478.php',
 'url': 'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.408478.php',
 'title': 'Videothek überfallen',
 'location': 'Charlottenburg-Wilmersdorf',
 'date': '2015-11-24',
 'nr': '2925',
 'text': 'Ziel eines Überfalls war in der vergangenen Nacht eine Videothek in Charlottenburg. Ermittlungen zufolge betrat kurz vor Mitternacht ein maskierter Mann das Geschäft in der Bismarckstraße, bedrohte den anwesenden Angestellten mit einer Schusswaffe und forderte die Herausgabe von Geld. Anschließend flüchtete der Räuber mit der erlangten Beute in unbekannte Richtung. Der 22-jährige Angestellte blieb unverletzt. Die Ermittlungen führt das örtlich zuständige Raubkommissariat der Polizeidirektion 2.',
 'words': ['ziel',
  'überfalls',
  'nacht',
  'videothek',
  'charlottenburg',
  'ermittlungen',
  'betrat',
  'mitternacht',
  'maskierter',
  'mann',
  'geschäft',
  'bismarckstraße',
  'bedrohte',


In [7]:
tfidf.getSimilarArticles(test_url2, n_results=5, return_sims=True, min_sim=0.2)

TF-IDF module: Input TF-IDF calculation: word {'angestellten'} in query is not in db
TF-IDF module: Input TF-IDF calculation: word {'anwesenden'} in query is not in db
TF-IDF module: Input TF-IDF calculation: word {'betrat'} in query is not in db
TF-IDF module: Input TF-IDF calculation: word {'blieb'} in query is not in db
TF-IDF module: Input TF-IDF calculation: word {'erlangten'} in query is not in db
TF-IDF module: Input TF-IDF calculation: word {'flüchtete'} in query is not in db
TF-IDF module: Input TF-IDF calculation: word {'forderte'} in query is not in db
TF-IDF module: Input TF-IDF calculation: word {'führt'} in query is not in db
TF-IDF module: Input TF-IDF calculation: word {'zuständige'} in query is not in db
TF-IDF module: Input TF-IDF calculation: word {'überfalls'} in query is not in db


[('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.385758.php',
  0.6582476990997616),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.447319.php',
  0.6500764146491781),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.717756.php',
  0.6329743357896492),
 ('https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.338367.php',
  0.6315409973903732)]

In [3]:
list(dao.pagedetails_collection.find({}, {"lemma_words":1}))

[{'_id': 'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.247355.php',
  'lemma_words': ['überfall',
   'spielothek',
   'nacht',
   'neukölln',
   '4.30',
   'uhr',
   'betreten',
   'unbekannte',
   'gezogen',
   'schusswaffe',
   'spielhallenräume',
   'britzer',
   'damm',
   'bedrohen',
   '31',
   'jahr',
   'alt',
   'angestellte',
   'fordern',
   'geld',
   'gerangel',
   'räuber',
   'schuss',
   'abgeben',
   'getroffen',
   'täter',
   'versuchen',
   'flüchten',
   'mitarbeiter',
   'festhalten',
   'beißen',
   'kriminelle',
   'hand',
   'gelangen',
   'schließlich',
   'beute',
   'flucht',
   'jahnstraße',
   'angestellte',
   'biss',
   'verletzen',
   'raubkommissariat',
   'polizeidirektion',
   '5',
   'ermittlung',
   'übernehmen']},
 {'_id': 'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.247354.php',
  'lemma_words': ['fußgängerin',
   'gestern',
   'nachmittag',
   'unfall',
   'hermsdorf',
   'schwer',
   'verletzen',
   'ermit